# Data cleansing of the Jeopardy dataset
This will ensure the data can easily be ingested in SQL table

### Imports

In [69]:
import pandas as pd
import numpy as np

### Load data

In [70]:
df = pd.read_csv('../data/combined_season1-40.tsv', sep='\t')
df.shape

(515894, 9)

### Get column types

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515894 entries, 0 to 515893
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   round               515894 non-null  int64 
 1   clue_value          515894 non-null  int64 
 2   daily_double_value  515894 non-null  int64 
 3   category            515894 non-null  object
 4   comments            34130 non-null   object
 5   answer              515894 non-null  object
 6   question            515894 non-null  object
 7   air_date            515894 non-null  object
 8   notes               88325 non-null   object
dtypes: int64(3), object(6)
memory usage: 35.4+ MB


### Set data type of air date

In [72]:
#df['air_date'] = df['air_date'].astype('datetime64[D]')
#df.info()

### Replace nan with blank for MySQL

In [73]:
df = df.replace({np.nan:''})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515894 entries, 0 to 515893
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   round               515894 non-null  int64 
 1   clue_value          515894 non-null  int64 
 2   daily_double_value  515894 non-null  int64 
 3   category            515894 non-null  object
 4   comments            515894 non-null  object
 5   answer              515894 non-null  object
 6   question            515894 non-null  object
 7   air_date            515894 non-null  object
 8   notes               515894 non-null  object
dtypes: int64(3), object(6)
memory usage: 35.4+ MB


### Replace all non-alphanumerics from strings to ensure MySQL takes them without errors

In [74]:
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].str.replace(r'\\', r'', regex=True).str.replace(r"'", r'', regex=True).str.replace(r'"', r'', regex=True) 
        df[col] = df[col].str.replace('\n', '').str.replace('\r', '')

In [75]:
for col in df.columns:
    if df[col].dtype == object:
        print(f'{col}: ')
        single_back = df[df[col].str.contains(r'\\', regex=True)].shape
        single_quote = df[df[col].str.contains(r"'", regex=True)].shape
        double_quote = df[df[col].str.contains(r'"', regex=True)].shape
        print(f'Single backslash: {single_back}')
        print(f'Single quote: {single_quote}')
        print(f'Double quote: {double_quote}')


category: 
Single backslash: (0, 9)
Single quote: (0, 9)
Double quote: (0, 9)
comments: 
Single backslash: (0, 9)
Single quote: (0, 9)
Double quote: (0, 9)
answer: 
Single backslash: (0, 9)
Single quote: (0, 9)
Double quote: (0, 9)
question: 
Single backslash: (0, 9)
Single quote: (0, 9)
Double quote: (0, 9)
air_date: 
Single backslash: (0, 9)
Single quote: (0, 9)
Double quote: (0, 9)
notes: 
Single backslash: (0, 9)
Single quote: (0, 9)
Double quote: (0, 9)


### Save cleansed dataset

In [76]:
df.to_csv('../data/cleansed_jeopardy_set.tsv', index = False, sep='\t')

In [83]:
import mysql.connector
import pandas as pd
import os
from dotenv import load_dotenv
from mysql.connector import Error
from tqdm import tqdm

load_dotenv()  # Load from .env file

def create_connection():
    try:
        connection = mysql.connector.connect(
            host=os.getenv("DB_HOST"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            database = os.getenv("DB_NAME")
        )
        if connection.is_connected():
            print("Connection successful!")
        return connection
    except Error as e:
        print(f"Connection error: {e}")
        return None
# 1. Connect to the database
conn = create_connection()
cursor = conn.cursor()

# 2. Prepare your DataFrame
# Example DataFrame structure
# df = pd.read_csv('your_data.csv')

# 3. Write the INSERT statement
insert_sql = """
    INSERT INTO qa_pairs (round, clue_value, daily_double_value, category, comments, answer, question, air_date, notes)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# 4. Loop through DataFrame rows
for i, row in tqdm(df.iloc[last_i:,:].iterrows(), total = df.shape[0] - last_i):
    values = (
        row['round'],
        row['clue_value'],
        row['daily_double_value'],
        row['category'],
        row['comments'],
        row['answer'],
        row['question'],
        row['air_date'],
        row['notes']
    )
    cursor.execute(insert_sql, values)
    if i % 500 == 0:
        conn.commit()
        print(f'Committed: {i}')

# 5. Commit and close
conn.commit()
cursor.close()
conn.close()


Connection successful!


  0%|          | 504/300893 [00:25<4:31:50, 18.42it/s]

Committed: 215500


  0%|          | 1002/300893 [00:51<4:36:17, 18.09it/s]

Committed: 216000


...

100%|█████████▉| 300003/300893 [4:14:42<00:52, 16.94it/s]

Committed: 515000


100%|█████████▉| 300504/300893 [4:15:08<00:20, 18.62it/s]

Committed: 515500


100%|██████████| 300893/300893 [4:15:27<00:00, 19.63it/s]


In [82]:
last_i = 215001